In [46]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

**Preparing Dataset**

In [47]:
df = pd.read_csv("Dataset/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head()
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)

In [48]:
df['domain1_score']=temp['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",6.0
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7.0
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5.0
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8.0
4,5,1,"Dear @LOCATION1, I know having computers has a...",6.0


In [49]:
df['essay'][0]

"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econom

In [50]:
temp.head(1)

,essay,domain1_score,essay_id,essay_set,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,"Dear local newspaper, I think effects computer...",8,1,1,6,Dear local newspaper I think effects computer...,1441,344,16,4.188953,11,76,75,18,24


In [51]:
#Make Dataset
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [53]:
X_train.shape

(9083, 3)

**PREPROCESSING**

In [54]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [55]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)

In [57]:
len(train_sents)

116500

In [58]:
train_sents[0]

['It',
 'first',
 'day',
 'high',
 'school',
 'gut',
 'full',
 'butterflies',
 'make',
 'want',
 'run',
 'bathrooms',
 'hide',
 'world']

**Preparing WORD2VEC and LSTM Model**

In [75]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [76]:
#Training Word2Vec model
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 vector_size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

C:\Users\NasolPC\AppData\Local\Temp\ipykernel_1780\1462232201.py:15: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [77]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec, model.wv[i])      
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)

C:\Users\NasolPC\AppData\Local\Temp\ipykernel_1780\2001189974.py:9: RuntimeWarning: invalid value encountered in divide
  vec = np.divide(vec,noOfWords)


In [78]:
training_vectors.shape

(9083, 300)

In [79]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

d:\GitHub_repositories\pythonProject\LSTM\Automatic-Essay-Scoring\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 1, 300)         │       721,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 814,705 (3.11 MB)

 Trainable params: 814,705 (3.11 MB)

 Non-trainable params: 0 (0.00 B)

In [80]:
training_vectors.shape

(9083, 1, 300)


**TRAINING AND PREDICTION**







In [81]:
lstm_model.fit(training_vectors, y_train, batch_size=32, epochs=100)

Epoch 1/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: nan - mae: nan
Epoch 2/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: nan - mae: nan
Epoch 3/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: nan - mae: nan
Epoch 4/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: nan - mae: nan
Epoch 5/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: nan - mae: nan
Epoch 6/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: nan - mae: nan
Epoch 7/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: nan - mae: nan
Epoch 8/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: nan - mae: nan
Epoch 9/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: nan - mae: nan
Epoch 10/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: nan - mae: nan
Epoch 11/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: nan - mae: nan
Epoch 12/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: nan - mae: nan
Epoch 13/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: nan - mae:

KeyboardInterrupt: 

In [ ]:
# lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)